In [1]:
!pip install mrjob

In [43]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [6]:
import pandas as pd

# Reading the yelp.csv file
df = pd.read_csv("yelp.csv")

# Replacing newline characters with spaces in all the columns of the yelp.csv file
df = df.replace(r'\n', ' ', regex=True)

# Saving the modified DataFrame to a new CSV file which is new_yelp.csv file with a specific delimiter
df.to_csv("new_yelp.csv")

In [ ]:
TASK 1. Average number of words in each review 

In [55]:
%%file average_word_count.py

from mrjob.job import MRJob
import csv

class AverageWordCount(MRJob):

    def mapper(self, _, line):
        row = next(csv.reader([line]))
        id, business_id,date,review_id,stars,text,type,user_id,cool,useful,funny=row
        #business_id, date, review_id, stars, text, type, user_id, cool, useful, funny = line.strip().split('\t')
        
        # Split the text into words and yield the length of the words and store in word_count
        word_count = len(text.split())
        
        # Emit null as the key and the word_count as the value
        yield None, word_count
        

    def reducer(self, keys, word_count):
        
        # Calculate the average word count for each review
        l1=list(word_count)
        total_words = sum(l1)
        num_reviews = len(l1)
        if num_reviews>0:
            average_word_count = total_words / num_reviews
            
            # Emit the Average_Word_Count string as the key and the average_word_count as the value
            yield "Average_Word_Count", average_word_count

            
if __name__ == '__main__':
    AverageWordCount.run()


Writing average_word_count.py


In [56]:
import average_word_count

# Creating an instance of the AverageWordCount MRJob with the new_yelp.csv input file
mr_job = average_word_count.AverageWordCount(args=['new_yelp.csv'])

# Execute the MRJob and printing the results in the console
with mr_job.make_runner() as runner:
    runner.run()
    for key, value in mr_job.parse_output(runner.cat_output()):
        print(f"{key}: {value}")


No configs specified for inline runner


Average_Word_Count: 131.02659734026597


In [ ]:
TASK 2. Count of reviews by year-month 

In [48]:
%%file year_month_review_count.py

from mrjob.job import MRJob
import csv

class CountofReview(MRJob):

    def mapper(self, _, line):
        row = next(csv.reader([line]))
        id, business_id, date, review_id, stars, text, type, user_id, cool, useful, funny = row
        
        # Extracting the YYYY-MM format from date column and store in date_review_count
        date_review_count = date[:7]
        
        # Emit the date_review_count as the key and 1 as the value
        yield date_review_count, 1
        

    def reducer(self, date_review_count, values):
        total_count_of_reviews = sum(values)
        
        # Emit the date_review_count as the key and the total_count_of_reviews as the value
        yield date_review_count, total_count_of_reviews

if __name__ == '__main__':
    CountofReview.run()


Writing year_month_review_count.py


In [49]:
import year_month_review_count

# Creating an instance of the CountofReview MRJob with the new_yelp.csv input file
mr_job = year_month_review_count.CountofReview(args=['new_yelp.csv'])

# Execute the MRJob and printing the results in the console
with mr_job.make_runner() as runner:
    runner.run()
    for key, value in mr_job.parse_output(runner.cat_output()):
        print(f"{key}: {value}")


No configs specified for inline runner


2005-04: 1
2005-07: 2
2005-12: 1
2006-01: 6
2006-02: 9
2006-04: 2
2006-05: 1
2006-06: 5
2006-07: 2
2006-08: 9
2006-09: 4
2006-10: 5
2006-11: 6
2006-12: 6
2007-01: 14
2007-02: 20
2007-03: 42
2007-04: 8
2007-05: 23
2007-06: 12
2007-07: 35
2007-08: 29
2007-09: 26
2007-10: 23
2007-11: 28
2007-12: 25
2008-01: 46
2008-02: 48
2008-03: 47
2008-04: 53
2008-05: 65
2008-06: 76
2008-07: 80
2008-08: 75
2008-09: 59
2008-10: 79
2008-11: 66
2008-12: 71
2009-01: 108
2009-02: 79
2009-03: 126
2009-04: 101
2009-05: 101
2009-06: 67
2009-07: 95
2009-08: 98
2009-09: 113
2009-10: 101
2009-11: 78
2009-12: 104
2010-01: 154
2010-02: 148
2010-03: 168
2010-04: 148
2010-05: 154
2010-06: 118
2010-07: 160
2010-08: 201
2010-09: 150
2010-10: 144
2010-11: 147
2010-12: 160
2011-01: 239
2011-02: 216
2011-03: 263
2011-04: 263
2011-05: 229
2011-06: 230
2011-07: 236
2011-08: 266
2011-09: 193
2011-10: 204
2011-11: 203
2011-12: 249
2012-01: 304
2012-02: 219
2012-03: 259
2012-04: 265
2012-05: 275
2012-06: 272
2012-07: 281
2012-

In [ ]:
TASK 3. Average rating of any review marked ”cool” 

In [112]:
%%file avg_cool_rating.py

from mrjob.job import MRJob
import csv

class AverageCoolRating(MRJob):

    def mapper(self, _, line):
        row = next(csv.reader([line]))
        id, business_id, date, review_id, stars, text, type, user_id, cool, useful, funny = row
        total_stars=0
        count=0
        
        # Checking if cool is not equal to 0
        if cool!=0:
            #total_stars+=int(stars)
            #count+=1
            if stars!="stars":
                total_stars+=int(stars)
                count+=1
                
            # Emit null as the key and both total_stars,count as the value
            yield None, (total_stars,count)
            

    def reducer(self, keys, values):
        #l2=len(total_stars)
        total_rating = 0
        total_cool_count = 0
        for total_stars, count in values:
            total_rating += total_stars
            total_cool_count += count
        if total_cool_count>0:
            average_cool_rating = total_rating / total_cool_count
            
            # Emit the Average_Word_Count string as the key and the average_cool_rating as the value
            yield "Average_Word_Count", average_cool_rating
        #yield "Average", cool

if __name__ == '__main__':
    CountofReview.run()


Writing avg_cool_rating.py


In [113]:
import avg_cool_rating

# Creating an instance of the AverageCoolRating MRJob with the new_yelp.csv input file
mr_job = avg_cool_rating.AverageCoolRating(args=['new_yelp.csv'])

# Execute the MRJob and printing the results in the console
with mr_job.make_runner() as runner:
    runner.run()
    for key, value in mr_job.parse_output(runner.cat_output()):
        print(f"{key}: {value}")


No configs specified for inline runner


Average_Word_Count: 3.7775
